In [1]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import statsmodels.api as sm

# 1A

In [2]:
ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
ff5_month = ff5_month
mom_month = pdr.get_data_famafrench('F-F_Momentum_Factor', start='1927-01', end='2023-03')[0]
mom_month = mom_month

ff5_month = ff5_month.reset_index()
mom_month = mom_month.reset_index()
ff5mom_month = ff5_month.merge(mom_month, on='Date', how='left')
ff5mom_month = ff5mom_month.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_30429/1568211615.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_30429/1568211615.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_30429/1568211615.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read y

In [3]:
ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
ff5_daily = ff5_daily
mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]
mom_daily = mom_daily

ff5_daily = ff5_daily.reset_index()
mom_daily = mom_daily.reset_index()
ff5mom_daily = ff5_daily.merge(mom_daily, on='Date', how='left')
ff5mom_daily = ff5mom_daily.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_30429/3591460299.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_30429/3591460299.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]


In [4]:
ff5mom_month

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-07,-0.39,-0.41,-0.97,0.68,-1.18,0.27,0.90
1963-08,5.07,-0.80,1.80,0.36,-0.35,0.25,1.01
1963-09,-1.57,-0.52,0.13,-0.71,0.29,0.27,0.19
1963-10,2.53,-1.39,-0.10,2.80,-2.01,0.29,3.12
1963-11,-0.85,-0.88,1.75,-0.51,2.24,0.27,-0.74
...,...,...,...,...,...,...,...
2022-11,4.60,-2.67,1.38,6.01,3.11,0.29,-2.01
2022-12,-6.41,-0.16,1.32,0.09,4.19,0.33,4.52
2023-01,6.65,4.43,-4.05,-2.62,-4.53,0.35,-15.98


In [5]:
ff5mom_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012,-0.21
1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012,0.42
1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012,0.41
1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012,0.07
1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012,-0.45
...,...,...,...,...,...,...,...
2023-03-27,0.27,0.51,1.02,-0.28,0.35,0.016,0.82
2023-03-28,-0.17,-0.03,0.74,0.08,0.55,0.016,0.73
2023-03-29,1.39,-0.34,-0.50,-0.90,-0.54,0.016,-1.11


In [6]:
ff5mom_month.index = pd.to_datetime(ff5mom_month.index.to_timestamp())
ff5mom_month.index = ff5mom_month.index + pd.offsets.MonthEnd(0)

# 1B

In [7]:
# Calculate the rolling variance using a window of 22 trading days
rolling_variance_daily = ff5mom_daily.rolling(22, min_periods=22).var()

In [8]:
rolling_variance_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-03-27,1.331266,0.424642,1.242579,0.226139,0.321225,4.935065e-07,0.758005
2023-03-28,1.281044,0.427383,1.175384,0.225673,0.331312,3.463203e-07,0.716987
2023-03-29,1.369642,0.422826,1.175816,0.272304,0.336809,1.818182e-07,0.776145


# 1C

In [9]:
monthly_data = rolling_variance_daily.resample('M').last()

In [10]:
monthly_data.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

In [11]:
for factor in monthly_data.columns:
    # Calculate the lagged variance
    df = pd.DataFrame()
    df['lagged_variance'] = monthly_data[factor]
    df['var'] = monthly_data[factor]

    df['lagged_variance'] = df['lagged_variance'].shift()
    # Drop any rows with missing values
    df = df.dropna()

    # Create the design matrix for the regression
    X = sm.add_constant(df['lagged_variance'])

    # Fit the regression model
    model = sm.OLS(df['var'], X)
    results = model.fit()

    # Print the regression results for each factor
    print(f"Results for {factor}:")
    print(results.summary())
    print()

Results for Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:                    var   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     186.5
Date:                Tue, 16 May 2023   Prob (F-statistic):           6.81e-38
Time:                        20:06:49   Log-Likelihood:                -1500.5
No. Observations:                 716   AIC:                             3005.
Df Residuals:                     714   BIC:                             3014.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.

It seems to predict returns in all cases but RF, but that's a leftover and not actually a "factor" so yes, it predicts the returns in all cases.

In [12]:
monthly_data.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

In [13]:
monthly_data.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

In [26]:
for factor in monthly_data.columns:
    # Calculate the lagged variance
    df = pd.DataFrame()
    df['var'] = monthly_data[factor]
    df['returns'] = ff5mom_month[factor]
    df['returns'] = df['returns'].shift(-1)
    print(df)
    # Drop any rows with missing values
    df = df.dropna()

    # Create the design matrix for the regression
    X = sm.add_constant(df['var'])

    # Fit the regression model
    model = sm.OLS(df['returns'], X)
    results = model.fit()

    # Print the regression results for each factor
    print(f"Results for {factor}:")
    print(results.summary())
    print()

                 var  returns
Date                         
1963-07-31  0.223726     5.07
1963-08-31  0.086949    -1.57
1963-09-30  0.154223     2.53
1963-10-31  0.171368    -0.85
1963-11-30  1.493974     1.83
...              ...      ...
2022-11-30  3.182266    -6.41
2022-12-31  1.833809     6.65
2023-01-31  1.169051    -2.58
2023-02-28  1.116542     2.51
2023-03-31  1.462451      NaN

[717 rows x 2 columns]
Results for Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.005432
Date:                Tue, 16 May 2023   Prob (F-statistic):              0.941
Time:                        20:07:46   Log-Likelihood:                -2092.4
No. Observations:                 716   AIC:                             4189.
Df Residuals:

In [27]:
# Calculate the long-run average monthly variances
sigi = monthly_data
ci = np.array(list(monthly_data.mean(axis=0)))

# Calculate the weights for each factor
wi = ci / sigi

# Calculate the excess returns on the volatility-managed portfolio
volatility_managed_returns = wi.shift() * ff5mom_month

In [29]:
volatility_managed_returns

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-08-31,23.747326,-10.981421,15.762410,2.515628,-1.545590,inf,7.617299
1963-09-30,-18.921550,-4.441025,1.090686,-8.269661,1.752552,inf,2.314570
1963-10-31,17.190736,-7.598421,-0.946349,15.085914,-10.004081,0.173271,24.943665
1963-11-30,-5.197698,-3.265103,4.985029,-1.279979,3.652190,inf,-4.555054
...,...,...,...,...,...,...,...
2022-11-30,1.527708,-2.902824,0.674295,1.561768,0.912653,0.742589,-0.748517
2022-12-31,-2.110785,-0.186353,0.356726,0.022603,0.657924,0.375562,0.516916
2023-01-31,3.800056,7.441123,-1.623087,-1.080581,-1.447364,0.896228,-5.375048


# 1f

In [30]:
cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   ']

In [31]:
volatility_managed_returns.index = pd.to_datetime(volatility_managed_returns.index)

# Create a new index with only the year and month
new_index = volatility_managed_returns.index.strftime('%Y-%m')

# Assign the new index back to your dataframe or series
volatility_managed_returns.index = new_index

volatility_managed_returns = volatility_managed_returns.dropna(subset=cols)

In [32]:
volatility_managed_returns.index = pd.to_datetime(volatility_managed_returns.index)
volatility_managed_returns.index = volatility_managed_returns.index + pd.offsets.MonthEnd(0)

In [33]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5['Mkt-RF']))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     576.4
Date:                Tue, 16 May 2023   Prob (F-statistic):           7.74e-94
Time:                        20:09:15   Log-Likelihood:                -2538.3
No. Observations:                 716   AIC:                             5081.
Df Residuals:                     714   BIC:                             5090.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5149      0.316      1.629      0.1

In [21]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5[['Mkt-RF', 'SMB', 'HML']]))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.445
Method:                 Least Squares   F-statistic:                     191.9
Date:                Tue, 16 May 2023   Prob (F-statistic):           3.54e-91
Time:                        20:06:49   Log-Likelihood:                -2538.1
No. Observations:                 716   AIC:                             5084.
Df Residuals:                     712   BIC:                             5102.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4892      0.319      1.533      0.1

In [22]:
volatility_managed_returns

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1963-08-31,23.747326,-10.981421,15.762410,2.515628,-1.545590,inf,7.617299
1963-09-30,-18.921550,-4.441025,1.090686,-8.269661,1.752552,inf,2.314570
1963-10-31,17.190736,-7.598421,-0.946349,15.085914,-10.004081,0.173271,24.943665
1963-11-30,-5.197698,-3.265103,4.985029,-1.279979,3.652190,inf,-4.555054
1963-12-31,1.283602,-2.905950,-0.082965,0.050009,-0.119502,0.216589,1.589168
...,...,...,...,...,...,...,...
2022-11-30,1.527708,-2.902824,0.674295,1.561768,0.912653,0.742589,-0.748517
2022-12-31,-2.110785,-0.186353,0.356726,0.022603,0.657924,0.375562,0.516916
2023-01-31,3.800056,7.441123,-1.623087,-1.080581,-1.447364,0.896228,-5.375048


In [23]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.467
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     88.72
Date:                Tue, 16 May 2023   Prob (F-statistic):           1.63e-92
Time:                        20:06:49   Log-Likelihood:                -2524.7
No. Observations:                 716   AIC:                             5065.
Df Residuals:                     708   BIC:                             5102.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1675      0.535      0.313      0.7

In [24]:
capm_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month['Mkt-RF']))
capm_results = capm_model.fit()
capm_alpha = capm_results.params[0]

# FF3 Model
ff3_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month[['Mkt-RF', 'SMB', 'HML']]))
ff3_results = ff3_model.fit()
ff3_alpha = ff3_results.params[0]

# FF5+Mom Model
ff5_mom_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month))
ff5_mom_results = ff5_mom_model.fit()
ff5_mom_alpha = ff5_mom_results.params[0]


# Print the alphas
print("CAPM: ", capm_results.summary())
print("FF3:", ff3_results.summary())
print("FF5+Mom:", ff5_mom_results.summary())

NameError: name 'portfolio_returns' is not defined